In [ ]:
from ctypes import resize
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch
import torchvision
from torchvision import transforms
import pathlib
import pandas as pd
import numpy as np
import random
import json
import cv2
import os
import math
import glob
from tqdm import tqdm
import pandas as pd
import h5py
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from functools import partial
from typing import Any, Callable, List, Optional, Sequence

import torch
from torch import nn, Tensor
from torch.nn import functional as F

tr = torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np

#trainimages=np.load('/content/drive/My Drive/trainimages-fold1.npy')

#trainimages=np.load('/content/drive/My Drive/train_seg_images-fold1.npy')

trainlabels=np.load('/content/drive/My Drive/trainlabels-fold1.npy')

#trainlabels=np.load('/content/drive/My Drive/train_aug_labels-fold1.npy')

#testimages=np.load('/content/drive/My Drive/testimages-fold1.npy')

testlabels=np.load('/content/drive/My Drive/testlabels-fold1.npy')

print(trainimages.shape)
print(trainlabels.shape)
print(testimages.shape)
print(testlabels.shape)

In [ ]:
np.savetxt('/content/drive/My Drive/train_gwo.csv', x_train, delimiter=',', fmt='%f')
np.savetxt('/content/drive/My Drive/test_gwo.csv', x_valid, delimiter=',', fmt='%f')

In [ ]:
x_train = pd.read_csv("/content/drive/My Drive/train_gwo.csv", header=None)

x_train.shape

x_train = np.array(x_train,dtype = 'float16')


x_valid = pd.read_csv("/content/drive/My Drive/test_gwo.csv", header=None)

x_valid.shape

x_valid =np.array(x_valid,dtype = 'float16')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import cv2
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from tqdm import tqdm

import albumentations as A

import os
import matplotlib.pyplot as plt

IMG_PATH = '/content/drive/MyDrive/segmentation/ISIC2018_Task1-2_Training_Input'
#IMG_PATH = '/content/drive/MyDrive/segmentation/ISIC2018_Task1-2_Training_Input_Contrast_Enhanced'
MASK_PATH = '/content/drive/MyDrive/segmentation/ISIC2018_Task1_Training_GroundTruth'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE= 4
SIZE = 256
LEARNING_RATE = 0.0003
NUM_EPOCHS = 20
NUM_WORKERS = 2
PIN_MEMORY = True

class ISICDataset(Dataset):
    def __init__(self, images_path, masks_path, size, transform=None):
        self.images_path = images_path
        self.masks_path = masks_path
        self.transform = transform
        #print("images_path---------------------------> ",images_path)
        self.ids = [image_file[:-4] for image_file in os.listdir(images_path) if image_file.endswith('.jpg')]
        self.size = size

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        image_path = os.path.join(self.images_path, self.ids[idx] + '.jpg')

        mask_path = os.path.join(self.masks_path, self.ids[idx] + '_segmentation.png')

        # Load image and mask
        img = cv2.imread(os.path.join(self.images_path, self.ids[idx] + '.jpg'), cv2.IMREAD_COLOR)
        mask = cv2.imread(os.path.join(self.masks_path, self.ids[idx] + '_segmentation.png'), cv2.IMREAD_GRAYSCALE)

         # Convert to RGB, And convert mask to binary
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        ret, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

        if self.transform is not None:
            augmentations = self.transform(image=img, mask=mask)
            img = augmentations['image']
            mask = augmentations['mask']

        # Convert numpy arrays to PyTorch tensors
        img = torch.from_numpy(img).permute(2, 0, 1).float() / 255.
        mask = torch.from_numpy(mask).unsqueeze(0).float()
        #mask = torch.from_numpy(mask).float()
        mask[mask == 255.0] = 1.0

        return img, mask

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
transform = A.Compose(
[
    A.Resize(height=SIZE, width=SIZE),
    A.Rotate(limit=35, p=1.0),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.1)
])


dataset = ISICDataset(images_path=IMG_PATH,
                            masks_path=MASK_PATH, size=SIZE, transform=transform)

train_size = int(0.8 * len(dataset))
train_dataset = Subset(dataset, range(train_size))
test_dataset = Subset(dataset, range(train_size, len(dataset)))

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
len(test_loader)

519

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F


class Swish(nn.Module):
    def __init__(self, name=None):
        super().__init__()
        self.name = name

    def forward(self, x):
        return x * torch.sigmoid(x)


class Conv2dSamePadding(nn.Conv2d):
    """2D Convolutions with same padding
    """

    def __init__(self, in_channels, out_channels, kernel_size, stride=1, dilation=1, groups=1, bias=True, name=None):
        super().__init__(in_channels, out_channels, kernel_size, stride, padding=0, dilation=dilation, groups=groups,
                         bias=bias)
        self.stride = self.stride if len(self.stride) == 2 else [self.stride[0]] * 2
        self.name = name

    def forward(self, x):
        input_h, input_w = x.size()[2:]
        kernel_h, kernel_w = self.weight.size()[2:]
        stride_h, stride_w = self.stride
        output_h, output_w = math.ceil(input_h / stride_h), math.ceil(input_w / stride_w)
        pad_h = max((output_h - 1) * self.stride[0] + (kernel_h - 1) * self.dilation[0] + 1 - input_h, 0)
        pad_w = max((output_w - 1) * self.stride[1] + (kernel_w - 1) * self.dilation[1] + 1 - input_w, 0)
        if pad_h > 0 or pad_w > 0:
            x = F.pad(x, [pad_w // 2, pad_w - pad_w // 2, pad_h // 2, pad_h - pad_h // 2])
        return F.conv2d(x, self.weight, self.bias, self.stride, self.padding, self.dilation, self.groups)


class BatchNorm2d(nn.BatchNorm2d):
    def __init__(self, num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True, name=None):
        super().__init__(num_features, eps=eps, momentum=momentum, affine=affine,
                         track_running_stats=track_running_stats)
        self.name = name


def drop_connect(inputs, drop_connect_rate, training):
    if not training:
        return inputs
    batch_size = inputs.shape[0]
    keep_prob = 1.0 - drop_connect_rate
    random_tensor = keep_prob
    random_tensor += torch.rand([batch_size, 1, 1, 1], dtype=inputs.dtype, device=inputs.device)
    binary_tensor = torch.floor(random_tensor)
    output = inputs / keep_prob * binary_tensor
    return output


class MBConvBlock(nn.Module):
    """Mobile Inverted Residual Bottleneck Block
    """

    def __init__(self, block_args, global_params, idx):
        super().__init__()

        block_name = 'blocks_' + str(idx) + '_'

        self.block_args = block_args
        self.batch_norm_momentum = 1 - global_params.batch_norm_momentum
        self.batch_norm_epsilon = global_params.batch_norm_epsilon
        self.has_se = (self.block_args.se_ratio is not None) and (0 < self.block_args.se_ratio <= 1)
        self.id_skip = block_args.id_skip

        self.swish = Swish(block_name + '_swish')

        # Expansion phase
        in_channels = self.block_args.input_filters
        out_channels = self.block_args.input_filters * self.block_args.expand_ratio
        if self.block_args.expand_ratio != 1:
            self._expand_conv = Conv2dSamePadding(in_channels=in_channels,
                                                  out_channels=out_channels,
                                                  kernel_size=1,
                                                  bias=False,
                                                  name=block_name + 'expansion_conv')
            self._bn0 = BatchNorm2d(num_features=out_channels,
                                    momentum=self.batch_norm_momentum,
                                    eps=self.batch_norm_epsilon,
                                    name=block_name + 'expansion_batch_norm')

        # Depth-wise convolution phase
        kernel_size = self.block_args.kernel_size
        strides = self.block_args.strides
        self._depthwise_conv = Conv2dSamePadding(in_channels=out_channels,
                                                 out_channels=out_channels,
                                                 groups=out_channels,
                                                 kernel_size=kernel_size,
                                                 stride=strides,
                                                 bias=False,
                                                 name=block_name + 'depthwise_conv')
        self._bn1 = BatchNorm2d(num_features=out_channels,
                                momentum=self.batch_norm_momentum,
                                eps=self.batch_norm_epsilon,
                                name=block_name + 'depthwise_batch_norm')

        # Squeeze and Excitation layer
        if self.has_se:
            num_squeezed_channels = max(1, int(self.block_args.input_filters * self.block_args.se_ratio))
            self._se_reduce = Conv2dSamePadding(in_channels=out_channels,
                                                out_channels=num_squeezed_channels,
                                                kernel_size=1,
                                                name=block_name + 'se_reduce')
            self._se_expand = Conv2dSamePadding(in_channels=num_squeezed_channels,
                                                out_channels=out_channels,
                                                kernel_size=1,
                                                name=block_name + 'se_expand')

        # Output phase
        final_output_channels = self.block_args.output_filters
        self._project_conv = Conv2dSamePadding(in_channels=out_channels,
                                               out_channels=final_output_channels,
                                               kernel_size=1,
                                               bias=False,
                                               name=block_name + 'output_conv')
        self._bn2 = BatchNorm2d(num_features=final_output_channels,
                                momentum=self.batch_norm_momentum,
                                eps=self.batch_norm_epsilon,
                                name=block_name + 'output_batch_norm')

    def forward(self, x, drop_connect_rate=None):
        identity = x
        # Expansion and depth-wise convolution
        if self.block_args.expand_ratio != 1:
            x = self._expand_conv(x)
            x = self._bn0(x)
            x = self.swish(x)

        x = self._depthwise_conv(x)
        x = self._bn1(x)
        x = self.swish(x)

        # Squeeze and Excitation
        if self.has_se:
            x_squeezed = F.adaptive_avg_pool2d(x, 1)
            x_squeezed = self._se_expand(self.swish(self._se_reduce(x_squeezed)))
            x = torch.sigmoid(x_squeezed) * x

        x = self._bn2(self._project_conv(x))

        # Skip connection and drop connect
        input_filters, output_filters = self.block_args.input_filters, self.block_args.output_filters
        if self.id_skip and self.block_args.strides == 1 and input_filters == output_filters:
            if drop_connect_rate:
                x = drop_connect(x, drop_connect_rate=drop_connect_rate, training=self.training)
            x = x + identity
        return x


def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )


def up_conv(in_channels, out_channels):
    return nn.ConvTranspose2d(
        in_channels, out_channels, kernel_size=2, stride=2
    )


def custom_head(in_channels, out_channels):
    return nn.Sequential(
        nn.Dropout(),
        nn.Linear(in_channels, 512),
        nn.ReLU(inplace=True),
        nn.Dropout(),
        nn.Linear(512, out_channels)
    )

In [ ]:
from functools import partial
from typing import Any, Callable, List, Optional, Sequence
from typing import Callable, List, Optional, Sequence, Tuple, Union
import collections
import torch
from torch import nn, Tensor
from torch.nn import functional as F
from torch.hub import load_state_dict_from_url
from itertools import repeat


def _make_ntuple(x: Any, n: int) -> Tuple[Any, ...]:
    """
    Make n-tuple from input x. If x is an iterable, then we just convert it to tuple.
    Otherwise we will make a tuple of length n, all with value of x.
    reference: https://github.com/pytorch/pytorch/blob/master/torch/nn/modules/utils.py#L8
    Args:
        x (Any): input value
        n (int): length of the resulting tuple
    """
    if isinstance(x, collections.abc.Iterable):
        return tuple(x)
    return tuple(repeat(x, n))

def stochastic_depth(input: Tensor, p: float, mode: str, training: bool = True) -> Tensor:
    """
    Implements the Stochastic Depth from `"Deep Networks with Stochastic Depth"
    <https://arxiv.org/abs/1603.09382>`_ used for randomly dropping residual
    branches of residual architectures.
    Args:
        input (Tensor[N, ...]): The input tensor or arbitrary dimensions with the first one
                    being its batch i.e. a batch with ``N`` rows.
        p (float): probability of the input to be zeroed.
        mode (str): ``"batch"`` or ``"row"``.
                    ``"batch"`` randomly zeroes the entire input, ``"row"`` zeroes
                    randomly selected rows from the batch.
        training: apply stochastic depth if is ``True``. Default: ``True``
    Returns:
        Tensor[N, ...]: The randomly zeroed tensor.
    """
    # if not torch.jit.is_scripting() and not torch.jit.is_tracing():
    #     _log_api_usage_once(stochastic_depth)
    if p < 0.0 or p > 1.0:
        raise ValueError(f"drop probability has to be between 0 and 1, but got {p}")
    if mode not in ["batch", "row"]:
        raise ValueError(f"mode has to be either 'batch' or 'row', but got {mode}")
    if not training or p == 0.0:
        return input

    survival_rate = 1.0 - p
    if mode == "row":
        size = [input.shape[0]] + [1] * (input.ndim - 1)
    else:
        size = [1] * input.ndim
    noise = torch.empty(size, dtype=input.dtype, device=input.device)
    noise = noise.bernoulli_(survival_rate)
    if survival_rate > 0.0:
        noise.div_(survival_rate)
    return input * noise

class StochasticDepth(nn.Module):
    """
    See :func:`stochastic_depth`.
    """

    def __init__(self, p: float, mode: str) -> None:
        super().__init__()
        self.p = p
        self.mode = mode

    def forward(self, input: Tensor) -> Tensor:
        return stochastic_depth(input, self.p, self.mode, self.training)


    def __repr__(self) -> str:
        s = f"{self.__class__.__name__}(p={self.p}, mode={self.mode})"
        return s

class Permute(torch.nn.Module):
    """This module returns a view of the tensor input with its dimensions permuted.
    Args:
        dims (List[int]): The desired ordering of dimensions
    """

    def __init__(self, dims: List[int]):
        super().__init__()
        self.dims = dims

    def forward(self, x: Tensor) -> Tensor:
        return torch.permute(x, self.dims)

class FrozenBatchNorm2d(torch.nn.Module):
    """
    BatchNorm2d where the batch statistics and the affine parameters are fixed
    Args:
        num_features (int): Number of features ``C`` from an expected input of size ``(N, C, H, W)``
        eps (float): a value added to the denominator for numerical stability. Default: 1e-5
    """

    def __init__(
        self,
        num_features: int,
        eps: float = 1e-5,
    ):
        super().__init__()
        self.eps = eps
        self.register_buffer("weight", torch.ones(num_features))
        self.register_buffer("bias", torch.zeros(num_features))
        self.register_buffer("running_mean", torch.zeros(num_features))
        self.register_buffer("running_var", torch.ones(num_features))

    def _load_from_state_dict(
        self,
        state_dict: dict,
        prefix: str,
        local_metadata: dict,
        strict: bool,
        missing_keys: List[str],
        unexpected_keys: List[str],
        error_msgs: List[str],
    ):
        num_batches_tracked_key = prefix + "num_batches_tracked"
        if num_batches_tracked_key in state_dict:
            del state_dict[num_batches_tracked_key]

        super()._load_from_state_dict(
            state_dict, prefix, local_metadata, strict, missing_keys, unexpected_keys, error_msgs
        )

    def forward(self, x: Tensor) -> Tensor:
        # move reshapes to the beginning
        # to make it fuser-friendly
        w = self.weight.reshape(1, -1, 1, 1)
        b = self.bias.reshape(1, -1, 1, 1)
        rv = self.running_var.reshape(1, -1, 1, 1)
        rm = self.running_mean.reshape(1, -1, 1, 1)
        scale = w * (rv + self.eps).rsqrt()
        bias = b - rm * scale
        return x * scale + bias


    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.weight.shape[0]}, eps={self.eps})"



class ConvNormActivation(torch.nn.Sequential):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: Union[int, Tuple[int, ...]] = 3,
        stride: Union[int, Tuple[int, ...]] = 1,
        padding: Optional[Union[int, Tuple[int, ...], str]] = None,
        groups: int = 1,
        norm_layer: Optional[Callable[..., torch.nn.Module]] = torch.nn.BatchNorm2d,
        activation_layer: Optional[Callable[..., torch.nn.Module]] = torch.nn.ReLU,
        dilation: Union[int, Tuple[int, ...]] = 1,
        inplace: Optional[bool] = True,
        bias: Optional[bool] = None,
        conv_layer: Callable[..., torch.nn.Module] = torch.nn.Conv2d,
    ) -> None:

        if padding is None:
            if isinstance(kernel_size, int) and isinstance(dilation, int):
                padding = (kernel_size - 1) // 2 * dilation
            else:
                _conv_dim = len(kernel_size) if isinstance(kernel_size, Sequence) else len(dilation)
                kernel_size = _make_ntuple(kernel_size, _conv_dim)
                dilation = _make_ntuple(dilation, _conv_dim)
                padding = tuple((kernel_size[i] - 1) // 2 * dilation[i] for i in range(_conv_dim))
        if bias is None:
            bias = norm_layer is None

        layers = [
            conv_layer(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                dilation=dilation,
                groups=groups,
                bias=bias,
            )
        ]

        if norm_layer is not None:
            layers.append(norm_layer(out_channels))

        if activation_layer is not None:
            params = {} if inplace is None else {"inplace": inplace}
            layers.append(activation_layer(**params))
        super().__init__(*layers)
        self.out_channels = out_channels

        if self.__class__ == ConvNormActivation:
            warnings.warn(
                "Don't use ConvNormActivation directly, please use Conv2dNormActivation and Conv3dNormActivation instead."
            )


class Conv2dNormActivation(ConvNormActivation):
    """
    Configurable block used for Convolution2d-Normalization-Activation blocks.
    Args:
        in_channels (int): Number of channels in the input image
        out_channels (int): Number of channels produced by the Convolution-Normalization-Activation block
        kernel_size: (int, optional): Size of the convolving kernel. Default: 3
        stride (int, optional): Stride of the convolution. Default: 1
        padding (int, tuple or str, optional): Padding added to all four sides of the input. Default: None, in which case it will calculated as ``padding = (kernel_size - 1) // 2 * dilation``
        groups (int, optional): Number of blocked connections from input channels to output channels. Default: 1
        norm_layer (Callable[..., torch.nn.Module], optional): Norm layer that will be stacked on top of the convolution layer. If ``None`` this layer wont be used. Default: ``torch.nn.BatchNorm2d``
        activation_layer (Callable[..., torch.nn.Module], optional): Activation function which will be stacked on top of the normalization layer (if not None), otherwise on top of the conv layer. If ``None`` this layer wont be used. Default: ``torch.nn.ReLU``
        dilation (int): Spacing between kernel elements. Default: 1
        inplace (bool): Parameter for the activation layer, which can optionally do the operation in-place. Default ``True``
        bias (bool, optional): Whether to use bias in the convolution layer. By default, biases are included if ``norm_layer is None``.
    """

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: Union[int, Tuple[int, int]] = 3,
        stride: Union[int, Tuple[int, int]] = 1,
        padding: Optional[Union[int, Tuple[int, int], str]] = None,
        groups: int = 1,
        norm_layer: Optional[Callable[..., torch.nn.Module]] = torch.nn.BatchNorm2d,
        activation_layer: Optional[Callable[..., torch.nn.Module]] = torch.nn.ReLU,
        dilation: Union[int, Tuple[int, int]] = 1,
        inplace: Optional[bool] = True,
        bias: Optional[bool] = None,
    ) -> None:

        super().__init__(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            groups,
            norm_layer,
            activation_layer,
            dilation,
            inplace,
            bias,
            torch.nn.Conv2d,
        )



class LayerNorm2d(nn.LayerNorm):
    def forward(self, x: Tensor) -> Tensor:
        x = x.permute(0, 2, 3, 1)
        x = F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        x = x.permute(0, 3, 1, 2)
        return x


class CNBlock(nn.Module):
    def __init__(
        self,
        dim,
        layer_scale: float,
        stochastic_depth_prob: float,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = partial(nn.LayerNorm, eps=1e-6)

        self.block = nn.Sequential(
            nn.Conv2d(dim, dim, kernel_size=7, padding=3, groups=dim, bias=True),
            Permute([0, 2, 3, 1]),
            norm_layer(dim),
            nn.Linear(in_features=dim, out_features=4 * dim, bias=True),
            nn.GELU(),
            nn.Linear(in_features=4 * dim, out_features=dim, bias=True),
            Permute([0, 3, 1, 2]),
        )
        self.layer_scale = nn.Parameter(torch.ones(dim, 1, 1) * layer_scale)
        self.stochastic_depth = StochasticDepth(stochastic_depth_prob, "row")

    def forward(self, input: Tensor) -> Tensor:
        result = self.layer_scale * self.block(input)
        result = self.stochastic_depth(result)
        result += input
        return result


class CNBlockConfig:
    # Stores information listed at Section 3 of the ConvNeXt paper
    def __init__(
        self,
        input_channels: int,
        out_channels: Optional[int],
        num_layers: int,
    ) -> None:
        self.input_channels = input_channels
        self.out_channels = out_channels
        self.num_layers = num_layers

    def __repr__(self) -> str:
        s = self.__class__.__name__ + "("
        s += "input_channels={input_channels}"
        s += ", out_channels={out_channels}"
        s += ", num_layers={num_layers}"
        s += ")"
        return s.format(**self.__dict__)


class ConvNeXt(nn.Module):
    def __init__(
        self,
        block_setting: List[CNBlockConfig],
        stochastic_depth_prob: float = 0.0,
        layer_scale: float = 1e-6,
        num_classes: int = 1000,
        block: Optional[Callable[..., nn.Module]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
        **kwargs: Any,
    ) -> None:
        super().__init__()

        if not block_setting:
            raise ValueError("The block_setting should not be empty")
        elif not (isinstance(block_setting, Sequence) and all([isinstance(s, CNBlockConfig) for s in block_setting])):
            raise TypeError("The block_setting should be List[CNBlockConfig]")

        if block is None:
            block = CNBlock

        if norm_layer is None:
            norm_layer = partial(LayerNorm2d, eps=1e-6)

        layers: List[nn.Module] = []

        # Stem
        firstconv_output_channels = block_setting[0].input_channels
        # layers.append(
        #     Conv2dNormActivation(
        #         3,
        #         firstconv_output_channels,
        #         kernel_size=4,
        #         stride=4,
        #         padding=0,
        #         norm_layer=norm_layer,
        #         activation_layer=None,
        #         bias=True,
        #     )
        # )
        layers.append(
            Conv2dNormActivation(
                3,
                firstconv_output_channels,
                kernel_size=4,
                stride=2,
                padding=1,
                norm_layer=norm_layer,
                activation_layer=None,
                bias=True,
            )
        )

        total_stage_blocks = sum(cnf.num_layers for cnf in block_setting)
        stage_block_id = 0
        for cnf in block_setting:
            # Bottlenecks
            stage: List[nn.Module] = []
            for _ in range(cnf.num_layers):
                # adjust stochastic depth probability based on the depth of the stage block
                sd_prob = stochastic_depth_prob * stage_block_id / (total_stage_blocks - 1.0)
                stage.append(block(cnf.input_channels, layer_scale, sd_prob))
                stage_block_id += 1
            layers.append(nn.Sequential(*stage))
            if cnf.out_channels is not None:
                # Downsampling
                layers.append(
                    nn.Sequential(
                        norm_layer(cnf.input_channels),
                        nn.Conv2d(cnf.input_channels, cnf.out_channels, kernel_size=2, stride=2),
                    )
                )

        self.features = nn.Sequential(*layers)
        self.avgpool = nn.AdaptiveAvgPool2d(1)

        lastblock = block_setting[-1]
        lastconv_output_channels = (
            lastblock.out_channels if lastblock.out_channels is not None else lastblock.input_channels
        )
        self.classifier = nn.Sequential(
            norm_layer(lastconv_output_channels), nn.Flatten(1), nn.Linear(lastconv_output_channels, num_classes)
        )

        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.Linear)):
                nn.init.trunc_normal_(m.weight, std=0.02)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def _forward_impl(self, x: Tensor) -> Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = self.classifier(x)
        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)


def _convnext(
    block_setting: List[CNBlockConfig],
    stochastic_depth_prob: float,
    weights_url,
    progress: bool,
    **kwargs: Any,
):

    model = ConvNeXt(block_setting, stochastic_depth_prob=stochastic_depth_prob, **kwargs)
#     if weights_url is not None:
#         weights =  load_state_dict_from_url(weights_url, progress=progress)
#         model.load_state_dict(weights)

    return model



URL_DICT = {
    "tiny": "https://download.pytorch.org/models/convnext_tiny-983f1562.pth",
    "small": "https://download.pytorch.org/models/convnext_small-0c510722.pth",
    "base": "https://download.pytorch.org/models/convnext_base-6075fbad.pth",
    "large": "https://download.pytorch.org/models/convnext_large-ea097f82.pth"
}


def convnext_tiny(pretrained = False, progress = True, **kwargs):

    url = URL_DICT["tiny"]
    block_setting = [
        CNBlockConfig(96, 192, 3),
        CNBlockConfig(192, 384, 3),
        CNBlockConfig(384, 768, 9),
        CNBlockConfig(768, None, 3),
    ]
    stochastic_depth_prob = kwargs.pop("stochastic_depth_prob", 0.1)
    return _convnext(block_setting, stochastic_depth_prob, url, progress, **kwargs)

def convnext_large(pretrained = False, progress = True, **kwargs):

    url = URL_DICT["large"]
    block_setting = [
        CNBlockConfig(192, 384, 3),
        CNBlockConfig(384, 768, 3),
        CNBlockConfig(768, 1536, 27),
        CNBlockConfig(1536, None, 3),
    ]
    stochastic_depth_prob = kwargs.pop("stochastic_depth_prob", 0.1)
    return _convnext(block_setting, stochastic_depth_prob, url, progress, **kwargs)

In [ ]:
class AttentionBlock(nn.Module):
    def __init__(self, f_g, f_l, f_int):
        super().__init__()

        self.w_g = nn.Sequential(
                                nn.Conv2d(f_g, f_int,
                                         kernel_size=1, stride=1,
                                         padding=0, bias=True),
                                nn.BatchNorm2d(f_int)
        )

        self.w_x = nn.Sequential(
                                nn.Conv2d(f_l, f_int,
                                         kernel_size=1, stride=1,
                                         padding=0, bias=True),
                                nn.BatchNorm2d(f_int)
        )

        self.psi = nn.Sequential(
                                nn.Conv2d(f_int, 1,
                                         kernel_size=1, stride=1,
                                         padding=0,  bias=True),
                                nn.BatchNorm2d(1),
                                nn.Sigmoid(),
        )

        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        g1 = self.w_g(g)
        x1 = self.w_x(x)
        psi = self.relu(g1+x1)
        psi = self.psi(psi)

        return psi*x

Conv+

In [ ]:
def get_blocks_to_be_concat(model, x):
    features = {}
    def get_features(name):
        def hook(model, input, output):
            features[name] = output.detach()
        return hook

#     model[1][2].register_forward_hook(get_features('feat1'))
#     model[3][2].register_forward_hook(get_features('feat2'))
#     model[5][26].register_forward_hook(get_features('feat3'))

    model[1][2].register_forward_hook(get_features('feat1'))
    model[3][2].register_forward_hook(get_features('feat2'))
    model[5][8].register_forward_hook(get_features('feat3'))


    # make a forward pass to trigger the hooks
    x = model(x)
    # print (features["feat1"].size())
    # print (features["feat2"].size())
    # print (features["feat3"].size())

    return x, features


class ConvNextUnet(nn.Module):
    def __init__(self, encoder,n_channels = 1536, out_channels=1, concat_input=True):
        super().__init__()

        self.encoder = encoder.features
        print (self.encoder[1][2])
        self.concat_input = concat_input

        n_channels = 1536
        self.size = [1536, 768, 384, 192]

        self.up_conv1 = up_conv(n_channels, 768)
        self.att3 = AttentionBlock(f_g=768, f_l=768, f_int=384)
        self.double_conv1 = double_conv(self.size[0], 768)

        self.up_conv2 = up_conv(768, 384)
        self.att2 = AttentionBlock(f_g=384, f_l=384, f_int=192)
        self.double_conv2 = double_conv(self.size[1], 384)

        self.up_conv3 = up_conv(384, 192)
        self.att1 = AttentionBlock(f_g=192, f_l=192, f_int=96)
        self.double_conv3 = double_conv(self.size[2], 192)

        self.up_conv_input = up_conv(192, 96)
        self.double_conv_input = double_conv(99, 32)

        #self.up_conv_input = up_conv(192, 96)
        #self.att0 = AttentionBlock(f_g=96, f_l=96, f_int=16)
        #self.double_conv_input = double_conv(99, 32)

#         n_channels = 768
#         self.size = [768, 384, 192, 96]
#         self.up_conv1 = up_conv(n_channels, 384)
#         self.double_conv1 = double_conv(self.size[0], 384)

#         self.up_conv2 = up_conv(384, 192)
#         self.double_conv2 = double_conv(self.size[1], 192)

#         self.up_conv3 = up_conv(192, 96)
#         self.double_conv3 = double_conv(self.size[2], 96)

#         self.up_conv_input = up_conv(96, 32)
#         self.double_conv_input = double_conv(35, 16)

#         self.final_conv = nn.Conv2d(16, 1, kernel_size=1)
        self.final_conv = nn.Conv2d(32, 1, kernel_size=1)


    def forward(self, x):
        input_ = x
        x, blocks = get_blocks_to_be_concat(self.encoder, x)

        x = self.up_conv1(x)
        x3 = self.att3(g=x, x=blocks["feat3"])
        x = torch.cat([x, x3], dim=1)
        x = self.double_conv1(x)



        x = self.up_conv2(x)
        x2 = self.att2(g=x, x=blocks["feat2"])
        x = torch.cat([x, x2], dim=1)
        x = self.double_conv2(x)



        x = self.up_conv3(x)
        x1 = self.att1(g=x, x=blocks["feat1"])
        x = torch.cat([x, x1], dim=1)
        x = self.double_conv3(x)

        #x = self.up_conv_input(x)

        #x0 = self.att0(g=x, x=input_)

        #x = torch.cat([x, x0], dim=1)

        x = self.up_conv_input(x)
        x = torch.cat([x, input_], dim=1)
        x = self.double_conv_input(x)


        x = self.final_conv(x)

        return x



def get_encoder():
    model = convnext_large(pretrained = False)
    return model

def create_convnext_unet():
    encoder = get_encoder()
    model = ConvNextUnet(encoder)
    return model

In [ ]:
def create_efficient_net():
    # unet = get_efficientunet_b7(out_channels=1, concat_input=True, pretrained=True).cuda()
    unet = create_convnext_unet()
    return unet
def create_optimizer(model):

    base_lr = 1e-4
    no_weight_decay_on_bn = False
    params_to_update = []
    for name, param in model.named_parameters():
        if param.requires_grad:
            params_to_update.append(param)
    optimizer = torch.optim.AdamW(params_to_update,lr = base_lr)
    return optimizer

class CreateLoss(nn.Module):
    def __init__(self):
        super(CreateLoss, self).__init__()
        self.loss_criterion = nn.BCEWithLogitsLoss()

    def forward(self, res_target, masks_target):
        res_target = res_target.squeeze(1)
        loss_target = self.loss_criterion(res_target, masks_target)
        return loss_target

class AverageMeter:
    """
    Class for calculating average
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, num):
        self.val = val
        self.sum += val * num
        self.count += num
        self.avg = self.sum / self.count

def set_seeds():
    print ("Setting up seeds!")
    random.seed(0)
    np.random.seed(0)
    torch.manual_seed(0)
    torch.cuda.manual_seed(0)

def setup_cudnn(cudnn_benchmark, cudnn_deterministic):
    print ("Setting up cudnn!")
    torch.backends.cudnn.benchmark = cudnn_benchmark
    torch.backends.cudnn.deterministic = cudnn_deterministic

def MakeDir(path):
    if (not os.path.exists(path)):
        os.mkdir(path)
use_cuda = True
cuda_device_id = 0
cudnn_benchmark = False
cudnn_deterministic = False
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")
# working_dir = "/kaggle/working/"
# print(f"The current working directory is {working_dir}")
# MakeDir(working_dir + "/checkpoint_save/")
if (use_cuda):
    torch.cuda.set_device(cuda_device_id)
set_seeds()
setup_cudnn(cudnn_benchmark, cudnn_deterministic)

Setting up seeds!
Setting up cudnn!


In [ ]:
use_cuda = True
device_id = 0
# model_path = "./checkpoint_save/checkpoint_0020.pth"
model_path = "/kaggle/input/hubmap-convnext-checkpoint/checkpoint_0030.pth"
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

if (use_cuda):
    torch.cuda.set_device(device_id)


filepath

In [ ]:
def check_accuracy(loader, model, device="cuda"):
    num_correct = 0
    dice_score = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
            num_correct += (preds == y).sum()
            dice_score += (2 * (preds * y).sum()) / (preds + y).sum() + 1e-8

    print(f"Dice score: {dice_score/len(loader)}")
    model.train()

In [ ]:
check_accuracy(test_loader, model, device)

Load model

In [ ]:
use_cuda = True
device_id = 0
# model_path = "./checkpoint_save/checkpoint_0020.pth"
model_path = "/kaggle/input/hubmap-convnext-checkpoint/checkpoint_0030.pth"
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

if (use_cuda):
    torch.cuda.set_device(device_id)
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

if (use_cuda):
    torch.cuda.set_device(device_id)


model = create_efficient_net()
#model.load_state_dict(torch.load("/content/drive/MyDrive/convnextmodel_att_5.pt"))
model.eval()
model = model.to(device)


In [ ]:
from google.colab.patches import cv2_imshow

from skimage import data, io

train_seg_images=[]


test_seg_images=[]



for i in range(8012):

  print(i)

  #io.imshow(trainimages[i].astype('uint8'))
  #io.show()

  img = torch.from_numpy(trainimages[i]).permute(2, 0, 1).unsqueeze(0).float()/ 255.

  img = img.to(device)

  res = model(img.to(device))

  mask =res

#mask = mask.squeeze(0).permute(1, 2, 0).detach().cpu().numpy()

  mask = mask.squeeze().detach().cpu().numpy()



  _, mask = cv2.threshold(mask, thresh=0.5, maxval=1, type=cv2.THRESH_BINARY)


  mask = np.expand_dims(mask, axis=2)

  mask = np.repeat(mask, 3, axis=2)

  bgcopy =trainimages[i].copy()

  mask_ind = (mask == 0)

  bgcopy[mask_ind] = mask[mask_ind]

  #plt.imshow(bgcopy.astype('uint8'))

  #plt.show()

  train_seg_images.append(bgcopy.astype('uint8'))


train_seg_images = np.array(train_seg_images,dtype = 'float16')

np.save("/content/drive/My Drive/train_seg_images-fold1.npy", train_seg_images)

train_seg_images.shape

In [ ]:
from google.colab.patches import cv2_imshow

from skimage import data, io

train_seg_images=[]


test_seg_images=[]



for i in range(2003):

  print(i)

  #io.imshow(trainimages[i].astype('uint8'))
  #io.show()

  img = torch.from_numpy(testimages[i]).permute(2, 0, 1).unsqueeze(0).float()/ 255.

  img = img.to(device)

  res = model(img.to(device))

  mask =res

#mask = mask.squeeze(0).permute(1, 2, 0).detach().cpu().numpy()

  mask = mask.squeeze().detach().cpu().numpy()



  _, mask = cv2.threshold(mask, thresh=0.5, maxval=1, type=cv2.THRESH_BINARY)


  mask = np.expand_dims(mask, axis=2)

  mask = np.repeat(mask, 3, axis=2)

  bgcopy =testimages[i].copy()

  mask_ind = (mask == 0)

  bgcopy[mask_ind] = mask[mask_ind]

  #plt.imshow(bgcopy.astype('uint8'))

  #plt.show()

  test_seg_images.append(bgcopy.astype('uint8'))


test_seg_images = np.array(test_seg_images,dtype = 'float16')

np.save("/content/drive/My Drive/test_seg_images-fold1.npy", test_seg_images)

test_seg_images.shape

In [ ]:
import numpy as np

trainimages=np.load('/content/drive/My Drive/trainimages-fold1.npy')

#trainimages=np.load('/content/drive/My Drive/skin dataset/train_aug_images-fold1.npy')

trainlabels=np.load('/content/drive/My Drive/trainlabels-fold1.npy')

#trainlabels=np.load('/content/drive/My Drive/skin dataset/train_aug_labels-fold1.npy')

testimages=np.load('/content/drive/My Drive/testimages-fold1.npy')
testlabels=np.load('/content/drive/My Drive/testlabels-fold1.npy')

print(trainimages.shape)
print(trainlabels.shape)
print(testimages.shape)
print(testlabels.shape)

In [ ]:
def create_scheduler(optimizer):
    milestones = [40,60,90]
    lr_decay = 0.1
    scheduler = torch.optim.lr_scheduler.MultiStepLR(
        optimizer,
        milestones=milestones,
        gamma=lr_decay)
    return scheduler

In [ ]:
loss_function = CreateLoss()
optimizer = create_optimizer(model)
scheduler = create_scheduler(optimizer)

show segmented Images

In [ ]:

for step, sample in enumerate(test_loader):

        #print ("step Number => " + str(step))

        images, masks = sample

        images = images.to(device)

        masks = masks.to(device)

        res = model(images)
        mask =res



        image = images[0].permute(1, 2, 0).detach().cpu().numpy()

        mask = mask[0].permute(1, 2, 0).detach().cpu().numpy()

        actualmask = masks[0].permute(1, 2, 0).detach().cpu().numpy()


        mask = cv2.resize(mask, (image.shape[1], image.shape[0]))
        mask = np.expand_dims(mask, axis=2)
        mask = np.repeat(mask, 3, axis=2)

        plt.figure(figsize = (20, 20))

        plt.subplot(1,3,1)
        plt.imshow(image)
        plt.title('Original Image',fontsize = 20)
        plt.axis('off')

        plt.subplot(1,3,2)
        plt.imshow(actualmask)
        plt.title('Target Image',fontsize = 20)
        plt.axis('off')
        plt.subplot(1,3,3)
        plt.imshow(mask)
        plt.title('Generated Image',fontsize = 20)
        plt.axis('off')
        plt.axis('off')
        plt.show()

In [ ]:
def check_accuracy(loader, model, device="cuda"):
    num_correct = 0
    dice_score = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
            num_correct += (preds == y).sum()
            dice_score += (2 * (preds * y).sum()) / (preds + y).sum() + 1e-8

    print(f"Dice score: {dice_score/len(loader)}")
    model.train()

def accuracyfunc(y_true, y_pred):
    '''Calculates the mean accuracy rate across all predictions for binary
    classification problems.
    '''
    return K.mean(K.equal(y_true, K.round(y_pred)))
def jaccard_distance(y_true, y_pred, smooth=100):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.square(y_true), axis = -1) + K.sum(K.square(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac)

def precisionfunc(y_true, y_pred):
    '''Calculates the precision, a metric for multi-label classification of
    how many selected items are relevant.
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recallfunc(y_true, y_pred):
    '''Calculates the recall, a metric for multi-label classification of
    how many relevant items are selected.
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

from keras import backend as K
def check_accuracy2(loader, model, device="cuda"):
    num_correct = 0
    recall = 0
    accuracy = 0
    precision= 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            #print("raw predict",model(x))
            preds = torch.sigmoid(model(x))
            #num_correct += (preds == y).sum()




            preds = (preds > 0.5).float()
            #print("preds",preds)
            #print("y",y)

            #num_correct += (preds == y).sum()
            prediction = preds.detach().cpu().numpy()
            label= y.detach().cpu().numpy()
            print("result --------------------->",recallfunc(label,prediction))
            recall += recallfunc(label,prediction)
            accuracy += accuracyfunc(label,prediction)
            precision+=precisionfunc(label,prediction)


    print(f"recall: {recall/len(loader)}")
    print(f"accuracy: {accuracy/len(loader)}")
    print(f"precision: {precision/len(loader)}")
    model.train()

In [ ]:
criterion = nn.BCEWithLogitsLoss()
def train(epoch,model,optimizer,scheduler,loss_function,train_loader):
    print ("Training!")
    use_cuda = True
    model.train()
    device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")
    Tensor = torch.cuda.FloatTensor if (use_cuda and torch.cuda.is_available()) else torch.FloatTensor

    acc = 0.	# Accuracy
    SE = 0.		# Sensitivity (Recall)
    SP = 0.		# Specificity
    PC = 0. 	# Precision
    F1 = 0.		# F1 Score
    JS = 0.		# Jaccard Similarity
    DC = 0.		# Dice Coefficient
    length = 0

    Train_Loss_meter = AverageMeter()
    for step, sample in enumerate(train_loader):

        #print ("step Number => " + str(step))


        images, masks = sample

        images = images.to(device)
        masks = masks.to(device)
        optimizer.zero_grad()
        res = model(images)

        #.unsqueeze(0)
        loss = criterion(res, masks)

        print ("Loss : " + str(loss))



        mask=res


        image = images[0].permute(1, 2, 0).detach().cpu().numpy()

        mask = mask[0].permute(1, 2, 0).detach().cpu().numpy()

        actualmask = masks[0].permute(1, 2, 0).detach().cpu().numpy()


        #mask = cv2.resize(mask, (image.shape[1], image.shape[0]))
        #mask = np.expand_dims(mask, axis=2)
        #mask = np.repeat(mask, 3, axis=2)



        loss.backward()
        optimizer.step()

        num = res.size(0)
        Train_Loss_meter.update(loss.item(), num)

    print ("Epoch : " + str(epoch) + " :: TrainLoss : " + str(Train_Loss_meter.avg))



In [ ]:
def validate(epoch,model,loss_function,train_loader):
    print ("Validating!")
    model.eval()
    use_cuda = True
    device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")
    Tensor = torch.cuda.FloatTensor if (use_cuda and torch.cuda.is_available()) else torch.FloatTensor

    Train_Loss_meter = AverageMeter()


    l = []
    for step, sample in enumerate(train_loader):
        images, masks = sample
        images = images.to(device)
        masks = masks.to(device)
        res = model(images)

        loss = criterion(res, masks)






        num = res.size(0)
        Train_Loss_meter.update(loss.item(), num)

    print ("Epoch : " + str(epoch) + " :: TrainLoss : " + str(Train_Loss_meter.avg))
    check_accuracy(test_loader, model, device)


In [ ]:
check_accuracy(test_loader, model, device)

Dice score: 0.8798908591270447


In [ ]:
check_accuracy2(test_loader, model, device)

In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:32'

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import torch
torch.cuda.empty_cache()

import gc
gc.collect()

# # Starting Training
# # -----------------------------------------
print ("Starting Training!")
epochs = 20
val_period = 1
checkpoint_period = 5
print ("Number of epochs : " + str(epochs))
print ("Validation Period : " + str(val_period))
print ("Checkpoint Period : " + str(checkpoint_period))
# # input("Press Enter to Start Training Model ?")
for epoch in range(1, epochs + 1):
     print ("Epoch Number => " + str(epoch))
     train(epoch,model,optimizer,scheduler,loss_function,train_loader)

     scheduler.step()
     if epoch % val_period == 0:
          validate(epoch, model, loss_function, test_loader)
     if (epoch % checkpoint_period == 0 or epoch == epochs):
        checkpoint_config = {'epoch': epoch}
        #checkpointer.save(f"checkpoint_{epoch:04d}", **checkpoint_config)
     filepath="/content/drive/MyDrive/convnextmodel_att_"+ str(epoch)+".pt"
     torch.save(model.state_dict(), filepath)
     print ("------------------------------------")

#check_accuracy2(test_loader, model, device)
check_accuracy(test_loader, model, device)

In [ ]:
filepath="/content/drive/MyDrive/convnextmodel.pt"
torch.save(model.state_dict(), filepath)

precion: **0.9102361798286438**


recall: 0.9072438478469849


accuracy: 0.9552687406539917

Dice score: 0.8995338678359985

Dice score: 0.8647632598876953
recall: 0.8940997123718262
accuracy: 0.9448225498199463
precision: 0.8728080987930298


> Indented block



In [ ]:
check_accuracy(test_loader, model, device)

In [ ]:
scaler = torch.cuda.amp.GradScaler()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scaler = torch.cuda.amp.GradScaler()

import torch
torch.cuda.empty_cache()

import gc
gc.collect()



for epoch in range(30):

    loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)

    if epoch % 5 == 0 and epoch != 0:
        checkpoint = {'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict(),
                      'epoch': epoch}
        torch.save(checkpoint, "checkpointN"+str(epoch)+"_.pth.tar")


    for batch_idx, (images, masks) in loop:
        images, masks = images.to(DEVICE), masks.to(DEVICE)

        # Gradients to 0
        optimizer.zero_grad()

        # Forward
        with torch.cuda.amp.autocast():
            outputs = model(images)
            loss = criterion(outputs, masks)

        # Backward
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        loop.set_description(f"Epoch[{epoch}/{NUM_EPOCHS}]")
        loop.set_postfix(loss = loss.item())
        checkpoint = {'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()}
        torch.save(checkpoint, "checkpoint_last.pth.tar")


In [ ]:
use_cuda = True
device_id = 0
# model_path = "./checkpoint_save/checkpoint_0020.pth"
model_path = "/kaggle/input/hubmap-convnext-checkpoint/checkpoint_0030.pth"
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

if (use_cuda):
    torch.cuda.set_device(device_id)

In [ ]:
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

if (use_cuda):
    torch.cuda.set_device(device_id)

model = create_efficient_net()
model = model.to(device)
# model.load_state_dict(torch.load(model_path)["model"])

In [ ]:
model.load_state_dict(torch.load(model_path)["model"])

In [ ]:
all_df_test = pd.read_csv("/kaggle/input/hubmap-organ-segmentation/test.csv")

In [ ]:
all_df_test.head(5)

In [ ]:
id_l = all_df_test["id"].values
image_width_l = all_df_test["img_width"].values

In [ ]:
id_l

In [ ]:
#https://www.kaggle.com/bguberfain/memory-aware-rle-encoding
#with transposed mask
def rle_encode_less_memory(img):
    #the image should be transposed
    pixels = img.T.flatten()

    # This simplified method requires first and last pixel to be zero
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]

    return ' '.join(str(x) for x in runs)

In [ ]:
names = []
preds = []
threshold = 0.5
for idx in tqdm(range(len(id_l))):
    id = id_l[idx]
    img_w = image_width_l[idx]

    image_path = "/kaggle/input/hubmap-organ-segmentation/test_images/" + str(id) + ".tiff"
    img = cv2.imread(image_path)
    img = cv2.resize(img, (512, 512))
    img = img/255.0
    img = torch.from_numpy(np.array(img))
    img = img.permute(2,0,1)
    img = img.unsqueeze(0)
    img = img.to(device).float()
    pred_mask = model(img)
    pred_mask = pred_mask.squeeze(1)
    pred_mask = pred_mask.cpu().data.numpy()
    pred_mask = cv2.resize(pred_mask, (img_w, img_w))
    pred_mask = pred_mask > threshold
    rle = rle_encode_less_memory(pred_mask)
    names.append(id)
    preds.append(rle)


In [ ]:
df = pd.DataFrame({'id':names,'rle':preds})
df.to_csv('submission.csv',index=False)

In [ ]:
df.head()